In [ ]:
import utils
import plotting
import postprocessing
import numpy as np
import warnings
import pandas as pd
from pandas.errors import SettingWithCopyWarning
from sample_labels import samples, sig_key, data_key
import os

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from PyPDF2 import PdfFileMerger

from copy import deepcopy

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
plot_dir = "../../../plots/BDTPreProcessing/Feb9"
_ = os.system(f"mkdir -p {plot_dir}")

In [ ]:
MAIN_DIR = "../../../"
samples_dir = f"{MAIN_DIR}/../data/skimmer/Jan31"
signal_samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb3"
year = "2017"

# Both Jet's Msds > 50 & at least one jet with Txbb > 0.8
filters = [
    [
        ("('ak8FatJetMsd', '0')", ">=", 50),
        ("('ak8FatJetMsd', '1')", ">=", 50),
        ("('ak8FatJetParticleNetMD_Txbb', '0')", ">=", 0.8),
    ],
    [
        ("('ak8FatJetMsd', '0')", ">=", 50),
        ("('ak8FatJetMsd', '1')", ">=", 50),
        ("('ak8FatJetParticleNetMD_Txbb', '1')", ">=", 0.8),
    ],
]

# # Both Jet's Msds > 50
# filters = [
#     [
#         ("('ak8FatJetMsd', '0')", ">=", 50),
#         ("('ak8FatJetMsd', '1')", ">=", 50),
#     ],
# ]

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()))

events_dict = utils.load_samples(signal_samples_dir, {sig_key: samples[sig_key]}, year, filters)
events_dict |= utils.load_samples(samples_dir, {k: samples[k] for k in samples.keys() - [sig_key]}, year, filters)

utils.add_to_cutflow(events_dict, "BDTPreselection", "weight", cutflow)

# print weighted sample yields
for sample in events_dict:
    tot_weight = np.sum(events_dict[sample]["weight"].values)
    print(f"Pre-selection {sample} yield: {tot_weight:.2f}")

In [ ]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)
postprocessing.derive_variables(events_dict, bb_masks)

In [ ]:
events = events_dict[sig_key]
postprocessing.postprocess_lpsfs(events, save_all=False)
events["finalWeight"] = events["finalWeight"] * events["VV_lp_sf_nom"]

In [ ]:
# {var: (bins, label)}
control_plot_vars = {
    "MET_pt": ([50, 0, 250], r"$p^{miss}_T$ (GeV)"),
    "DijetEta": ([50, -8, 8], r"$\eta^{jj}$"),
    "DijetPt": ([50, 0, 750], r"$p_T^{jj}$ (GeV)"),
    "DijetMass": ([50, 0, 2500], r"$m^{jj}$ (GeV)"),
    "bbFatJetEta": ([50, -3, 3], r"$\eta^{bb}$"),
    "bbFatJetPt": ([50, 200, 1000], r"$p^{bb}_T$ (GeV)"),
    "bbFatJetMsd": ([50, 20, 250], r"$m^{bb}$ (GeV)"),
    "bbFatJetParticleNetMD_Txbb": ([50, 0, 1], r"$p^{bb}_{Txbb}$"),
    "VVFatJetEta": ([50, -3, 3], r"$\eta^{VV}$"),
    "VVFatJetPt": ([50, 200, 1000], r"$p^{VV}_T$ (GeV)"),
    "VVFatJetMsd": ([50, 20, 500], r"$m^{VV}$ (GeV)"),
    "VVFatJetParticleNet_Th4q": ([50, 0, 1], r"Probability($H \to 4q$)"),
    "VVFatJetParTMD_THWW4q": ([50, 0, 1], r"Probability($H \to VV \to 4q$)"),
    "bbFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{bb}_T / p_T^{jj}$"),
    "VVFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{VV}_T / p_T^{jj}$"),
    "VVFatJetPtOverbbFatJetPt": ([50, 0.4, 2.5], r"$p^{VV}_T / p^{bb}_T$"),
    # "BDTScore": ([50, 0, 1], r"BDT Score"),
}

postprocessing.control_plots(events_dict, bb_masks, control_plot_vars, plot_dir)

In [28]:
BDT_samples = list(samples.keys())

BDT_data_vars = [
    "MET_pt",
    "DijetEta",
    "DijetPt",
    "DijetMass",
    "bbFatJetPt",
    "VVFatJetEta",
    "VVFatJetPt",
    "VVFatJetMsd",
    "VVFatJetParTMD_THWW4q",
    "VVFatJetParTMD_probQCD",
    "VVFatJetParTMD_probHWW3q",
    "VVFatJetParTMD_probHWW4q",
    "bbFatJetParticleNetMD_Txbb",
    "bbFatJetPtOverDijetPt",
    "VVFatJetPtOverDijetPt",
    "VVFatJetPtOverbbFatJetPt",
    "finalWeight",
]

bdt_events_dict = []

for sample in BDT_samples:
    events = pd.DataFrame(
        {var: utils.get_feat(events_dict[sample], var, bb_masks[sample]) for var in BDT_data_vars}
    )
    events["Dataset"] = sample
    bdt_events_dict.append(events)

bdt_events = pd.concat(bdt_events_dict, axis=0)

import pyarrow.parquet as pq
import pyarrow as pa

table = pa.Table.from_pandas(bdt_events)
pq.write_table(table, f"{samples_dir}/bdt_data.parquet")
